In [1]:
import re


def load_data(path):
    with open(path) as f:
        data = f.read()
    return data


def parse_data(data):
    a, b, c, program = re.findall(r'Register A: (\d+)\nRegister B: (\d+)\nRegister C: (\d+)\n\nProgram: ([,\d]*)', data)[0]
    a, b, c = int(a), int(b), int(c)
    program = [int(i) for i in program.split(',')]
    return a, b, c, program


def find_next_a(set_current_a, program_value):
    set_next_a = set()
    if not set_current_a:
        set_current_a = {0}
    for current_a in set_current_a:
        for i in range(8):
            next_a = current_a+i
            output = solve(next_a, b, c, program)
            if output == program[-len(output):]:
                set_next_a.add(next_a * 8)
    return set_next_a


def find_a(program):
    set_possible_a = set()
    for r_program_value in program[::-1]:
        set_possible_a = find_next_a(set_possible_a, r_program_value)
    return min(set_possible_a)//8


def solve(a, b, c, program):
    output = []
    pointer = 0
    while pointer <= len(program) - 1:
        opcode = program[pointer]
        if opcode == 4:
            b ^= c
        elif pointer <= len(program) - 1:
            literal_operand = program[pointer+1]
            if literal_operand == 4:
                combo_operand = a
            elif literal_operand == 5:
                combo_operand = b
            elif literal_operand == 6:
                combo_operand = c
            else:
                combo_operand = literal_operand
            if opcode == 0:
                a //= 2**combo_operand
            elif opcode == 1:
                b ^= literal_operand
            elif opcode == 2:
                b = combo_operand % 8
            elif opcode == 3 and a != 0:
                pointer = literal_operand
                continue
            elif opcode == 5:
                output.append(combo_operand % 8)
            elif opcode == 6:
                b = a // 2**combo_operand
            elif opcode == 7:
                c = a // 2**combo_operand
        pointer += 2
    return output


data = load_data('input.txt')
a, b, c, program = parse_data(data)
print(','.join([str(i) for i in solve(a, b, c, program)]))
print(','.join([str(i) for i in solve(find_a(program), b, c, program)]))

4,3,2,6,4,5,3,2,4
2,4,1,1,7,5,1,5,4,5,0,3,5,5,3,0
